In [5]:


import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

import pickle
import numpy as np
import math
from Camera import Camera
from Frame import Frame
import matplotlib.pyplot as plt
import Utils
%matplotlib qt


# generate dictionary fo gassian runs
from Frame import Frame

path = 'I:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov9_cornell/'
dict_name = f'{path}/dict/frames_model_cornell_2.pkl'
frame_start = 1741
frame_end = 1742
frames = list(range(frame_start,frame_end,1))
image_name= []
for frame in frames:
    image_name += [f'P{frame}CAM{cam + 1}' for cam in range(4)]

frames = {f'{im_name}.jpg':Frame(path,im_name,idx, camera_path = path,delta_xy = 80, use_mask = False,kernel = np.ones((2, 3))) for idx,im_name in enumerate(image_name)}


In [6]:
# generate dictionary fo gassian runs

frames_dict = {}
frames_numbers = list(range(frame_start,frame_end,1))
for frame_number in frames_numbers:
    cams,base_images,frame_mov  = {},{},{}
    for idx in range(4):
        frame_name = f'P{frame_number}CAM{idx+1}.jpg'
        frame = frames[frame_name]
        base_images[frame.image_id] = frame.generate_base_image()
        frame.save_croped_images()
        cams[frame.camera_number] = frame.cams_for_gs()
        if idx == 0:
            rot_z = frames[frame_name].rotation_matrix_from_vectors(frames[frame_name].R[2,:], [0,0,1])
        cams[frame.camera_number]['ew_to_lab'] = rot_z
        cams[frame.camera_number]['bounding_box'] = frame.bounding_box
        frame_mov['image_path'] = f'{frame.path}/images/'
        frame_mov['real_frames'] = frames_numbers

    frames_dict[frame_number] = [base_images,cams,[],frame_mov]

Utils.pickle_file(frames_dict,dict_name )